# Transformers, what can they do?

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers

# Just using Pipelines

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

In [ ]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445963859558105, 0.111976258456707, 0.043427448719739914]}

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

[{'generated_text': 'In this course, we will teach you how to understand and use '
                    'data flow and data interchange when handling user data. We '
                    'will be working with one or more of the most commonly used '
                    'data flows — data flows of various types, as seen by the '
                    'HTTP'}]

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

[{'generated_text': 'In this course, we will teach you how to manipulate the world and '
                    'move your mental and physical capabilities to your advantage.'},
 {'generated_text': 'In this course, we will teach you how to become an expert and '
                    'practice realtime, and with a hands on experience on both real '
                    'time and real'}]

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

[{'sequence': 'This course will teach you all about mathematical models.',
  'score': 0.19619831442832947,
  'token': 30412,
  'token_str': ' mathematical'},
 {'sequence': 'This course will teach you all about computational models.',
  'score': 0.04052725434303284,
  'token': 38163,
  'token_str': ' computational'}]

In [ ]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

[{'entity_group': 'PER', 'score': 0.99816, 'word': 'Sylvain', 'start': 11, 'end': 18}, 
 {'entity_group': 'ORG', 'score': 0.97960, 'word': 'Hugging Face', 'start': 33, 'end': 45}, 
 {'entity_group': 'LOC', 'score': 0.99321, 'word': 'Brooklyn', 'start': 49, 'end': 57}
]

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

{'score': 0.6385916471481323, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

[{'summary_text': ' America has changed dramatically during recent years . The '
                  'number of engineering graduates in the U.S. has declined in '
                  'traditional engineering disciplines such as mechanical, civil '
                  ', electrical, chemical, and aeronautical engineering . Rapidly '
                  'developing economies such as China and India, as well as other '
                  'industrial countries in Europe and Asia, continue to encourage '
                  'and advance engineering .'}]

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

[{'translation_text': 'This course is produced by Hugging Face.'}]

# AutoTokenizer and Automodel

In [20]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device : ", device)

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
inputs = tokenizer("I've been waiting for a HuggingFace course my whole life.", padding=True, truncation=True, return_tensors="pt")
inputs.to(device)
print("Input shape is 1 X number of tokens : ", inputs["input_ids"].shape)
#print(inputs["input_ids"])

# AutoModel is a general model that can pull any model from huggingface hub
# In this case we are pulling a bert which will output embeddings for each token
model = AutoModel.from_pretrained(checkpoint)
model.to(device)
outputs = model(**inputs)
print("Output last hidden state is one vector of embodding per token :", outputs.last_hidden_state.shape)

# Let's pull a model for sequence classification. So it will basically add
# a head with two outputs one for positive and one for negative. This does
# not have a hidden state anymore

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.to(device)
outputs = model(**inputs)
print("Outputs : ", outputs.logits.shape)

# last output is logits which is basically not actual values but log of actual
# values so we can softmax to convert to probabilites. Let's leave that

Device :  cuda
Input shape is 1 X number of tokens :  torch.Size([1, 16])
Output last hidden state is one vector of embodding per token : torch.Size([1, 16, 768])
Outputs :  torch.Size([1, 2])


# Tokenizers

In [60]:
# Small lesson on tokenizers
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)


sentences = [
    "yo hahah",
    "a red fox jumps over the wall and eats your apple"
]

batch = []
for sentence in sentences:
  # Instead of using tokenizer() directly let's break it down to see what all goes
  # behind it
  inputs = tokenizer.tokenize(sentence)
  print(inputs)

  # convert to numbers
  input_ids  = tokenizer.convert_tokens_to_ids(inputs)
  print(input_ids)
  batch.append(input_ids)

  # this is what goes in the model
  # notice how we convert it into a batch because that is what a model takes
  actual_input = torch.tensor([input_ids])
  print("Input IDs:", actual_input)

  # this will work on the batch of tensors
  output = model(actual_input)
  print(output.logits)
  print("=============")

# Batching, truncation and attention mask
# batch[0] is smaller than batch[1] make it equal length
original_length = len(batch[0])
batch[0] = batch[0] + [tokenizer.pad_token_id] * (len(batch[1]) - len(batch[0]))
print(batch)
batched = torch.tensor(batch)
print(model(batched).logits)

# the output for the first sentence in the batch is different this is because of
# attention mask
attention_mask = torch.ones(batched.shape[0], batched.shape[1])
attention_mask[0, original_length:] = 0
print("\nAfter attention mask")
print(model(batched, attention_mask=attention_mask).logits)


# When we pass the input to tokenizer directly it can do all the things
# like padding and truncate and attention mask
print("\nUsing tokenizer directly")
input = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
print(input['input_ids'])
print(input['attention_mask'])

['yo', 'ha', '##ha', '##h']
[10930, 5292, 3270, 2232]
Input IDs: tensor([[10930,  5292,  3270,  2232]])
tensor([[ 1.8348, -1.5315]], grad_fn=<AddmmBackward0>)
['a', 'red', 'fox', 'jumps', 'over', 'the', 'wall', 'and', 'eats', 'your', 'apple']
[1037, 2417, 4419, 14523, 2058, 1996, 2813, 1998, 20323, 2115, 6207]
Input IDs: tensor([[ 1037,  2417,  4419, 14523,  2058,  1996,  2813,  1998, 20323,  2115,
          6207]])
tensor([[-1.6322,  1.7704]], grad_fn=<AddmmBackward0>)
[[10930, 5292, 3270, 2232, 0, 0, 0, 0, 0, 0, 0], [1037, 2417, 4419, 14523, 2058, 1996, 2813, 1998, 20323, 2115, 6207]]
tensor([[ 0.6134, -0.6137],
        [-1.6322,  1.7704]], grad_fn=<AddmmBackward0>)

After attention mask
tensor([[ 1.8348, -1.5315],
        [-1.6322,  1.7704]], grad_fn=<AddmmBackward0>)

Using tokenizer directly
tensor([[  101, 10930,  5292,  3270,  2232,   102,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1037,  2417,  4419, 14523,  2058,  1996,  2813,  1998, 20323,
   